In [20]:
import galois
import numpy as np

# Define GF(2^2) as GF(4) using the irreducible polynomial x^2 + x + 1
GF4 = galois.GF(2**2, irreducible_poly="x^2 + x + 1", repr="poly")

# Define a helper function to convert GF(4) elements into symbolic form
def to_symbolic_string(value):
    """
    Converts a GF(4) element into its symbolic representation.
    """
    if value == GF4(0):
        return "0"
    elif value == GF4(1):
        return "1"
    elif value == GF4(2):
        return "x"
    elif value == GF4(3):
        return "x + 1"

# Define the functions F_0(x0, x1) and F_1(x0, x1)
def F_0(x0, x1):
    """Computes F_0(x0, x1) = x0 * x1 in GF(2)."""
    x0, x1 = GF4(x0), GF4(x1)  # Ensure inputs are GF(4) elements
    return x0 * x1  # Multiplication in GF(4)

def F_1(x0, x1):
    """Computes F_1(x0, x1) = x0 + x1 + 1 in GF(2)."""
    x0, x1 = GF4(x0), GF4(x1)  # Ensure inputs are GF(4) elements
    return x0 + x1 + GF4(1)  # Addition is XOR in GF(2)

# General c-derivative formula
def compute_c_derivative(F_xa, F_x, c0, c1):
    """
    Computes the c-derivative: D_c F(x, a) = F(x + a) + c * F(x).
    """
    x_a0, x_a1 = F_xa  # F(x + a)
    x0, x1 = F_x        # F(x)
    
    # Convert c0, c1 to GF4 elements
    c0 = GF4(c0)
    c1 = GF4(c1)
    
    # Compute F(x + a)
    Fx_a0 = F_0(x_a0, x_a1)
    Fx_a1 = F_1(x_a0, x_a1)

    # Compute c * F(x)
    c_Fx0 = c0 * F_0(x0, x1) + c1 * F_1(x0, x1)  # Addition in GF(4) (XOR in GF(2))
    c_Fx1 = c0 * F_1(x0, x1) + c1 * F_0(x0, x1) + c1 * F_1(x0, x1)  # Addition in GF(4)

    # Compute D_c F(x, a) = F(x + a) + c * F(x)
    D_c_F0 = Fx_a0 + c_Fx0  # Addition is XOR in GF(2)
    D_c_F1 = Fx_a1 + c_Fx1  # Addition is XOR in GF(2)

    return (D_c_F0, D_c_F1)

# Example inputs for F(x + a) and F(x)
a = (GF4(1), GF4(0))  # a = (1, 0)
x = (GF4("x"), GF4(1))  # x = (x, 1)
F_xa = (x[0] + a[0], x[1] + a[1])  # Compute x + a
F_x = x  # Original x

# Use numpy for iteration over c values
c_values = np.array([(c0, c1) for c0 in range(2) for c1 in range(2)])  # All (c0, c1) combinations

# Compute D_c F for all (c0, c1) combinations
results = {}
for c0, c1 in c_values:
    D_c_F = compute_c_derivative(F_xa, F_x, c0, c1)
    results[f"c = ({c0}, {c1})"] = D_c_F

# Display results in symbolic format
print("\n## Final Results (symbolic):")
for c_label, (expr1, expr2) in results.items():
    expr1_sym = to_symbolic_string(expr1)
    expr2_sym = to_symbolic_string(expr2)
    print(f"{c_label} : ({expr1_sym}, {expr2_sym})")



## Final Results (symbolic):
c = (0, 0) : (x + 1, x + 1)
c = (0, 1) : (1, x + 1)
c = (1, 0) : (1, 1)
c = (1, 1) : (x + 1, 1)
